In [2]:
import numpy as np
import dlib
import cv2

In [3]:
# 좌표체계 통일하기 위한 함수
def getFaceDimension(rect):
    return (rect.left(), rect.top(), rect.right() - rect.left(), rect.bottom() - rect.top())

In [4]:
# 얼굴과 눈 중심 좌표를 통해 crop, Margin 값 곱해줘서 영역 추가로 획득 가능
def getCropDimension(rect, center):
    width = (rect.right() - rect.left())
    half_width = width // 2
    (centerX, centerY) = center
    startX = centerX - half_width
    endX = centerX + half_width
    startY = rect.top()
    endY = rect.bottom() 
    return (startX, endX, startY, endY)    

In [5]:
# 눈만 찾음
RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))


# face_recognition을 위해 unknown dataset을 추가로 둠
dataset_paths = ['D:/opencv_dnn_202005/opencv_dnn_202005/dataset/tedy-front/', 
                 'D:/opencv_dnn_202005/opencv_dnn_202005/dataset/son-front/', 
                 'D:/opencv_dnn_202005/opencv_dnn_202005/dataset/unknown-front/']
output_paths = ['D:/opencv_dnn_202005/opencv_dnn_202005/dataset/tedy-align/',
                'D:/opencv_dnn_202005/opencv_dnn_202005/dataset/son-align/',
                'D:/opencv_dnn_202005/opencv_dnn_202005/dataset/unknown-align/']

number_images = 20
image_type = '.jpg'

predictor_file = 'D:/opencv_dnn_202005/opencv_dnn_202005/model/shape_predictor_68_face_landmarks.dat'
# 얼굴 detect 범위보다 좀 더 여유 공간 주기 위함
MARGIN_RATIO = 1.5
# crop 하기위한 이미지 사이즈
OUTPUT_SIZE = (300, 300)

# 얼굴을 찾아주는 detector
detector = dlib.get_frontal_face_detector()
# point를 찾아주는 predictor
predictor = dlib.shape_predictor(predictor_file)

# 파일마다 이미지 하나씩 읽어서 처리 시작
for (i, dataset_path) in enumerate(dataset_paths):
    output_path = output_paths[i]
    
    for idx in range(number_images):
        input_file = dataset_path + str(idx+1) + image_type

        # get RGB image from BGR, OpenCV format
        image = cv2.imread(input_file)
        image_origin = image.copy()

        (image_height, image_width) = image.shape[:2]
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # 얼굴 detect
        rects = detector(gray, 1)

        for (i, rect) in enumerate(rects):
            # 좌표체계 통일하기 위한 함수
            (x, y, w, h) = getFaceDimension(rect)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # 68개의 얼굴 특징점 좌표값
            points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
            # 68개의 얼굴 특징점 좌표값 중 눈만
            show_parts = points[EYES]

            # 눈 중심점 찾고, 각도 계산
            right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")
            left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")

            eye_delta_x = right_eye_center[0,0] - left_eye_center[0,0]
            eye_delta_y = right_eye_center[0,1] - left_eye_center[0,1]
            degree = np.degrees(np.arctan2(eye_delta_y,eye_delta_x)) - 180

            eye_distance = np.sqrt((eye_delta_x ** 2) + (eye_delta_y ** 2))
            aligned_eye_distance = left_eye_center[0,0] - right_eye_center[0,0]
            scale = aligned_eye_distance / eye_distance

            eyes_center = ((left_eye_center[0,0] + right_eye_center[0,0]) // 2,
                    (left_eye_center[0,1] + right_eye_center[0,1]) // 2)
                    
            metrix = cv2.getRotationMatrix2D(eyes_center, degree, scale)

            # 기울어진 각도 조정(앞의 이미지 alignment와 동일)
            warped = cv2.warpAffine(image_origin, metrix, (image_width, image_height),
                flags=cv2.INTER_CUBIC)

            # 이미지 crop
            (startX, endX, startY, endY) = getCropDimension(rect, eyes_center)

            croped = warped[startY:endY, startX:endX]
            output = cv2.resize(croped, OUTPUT_SIZE)
            #output = warped[startY:endY, startX:endX]
            
            # crop한 결과 저장
            output_file = output_path + str(idx+1) + image_type
            #cv2.imshow(output_file, output)
            cv2.imwrite(output_file, output)
        
cv2.waitKey(0)   
cv2.destroyAllWindows()